In [168]:
import numpy as np
import csv
import math

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

train_size = .9
K = 30
learn_rate = 0.0000001
iterations = 10
w = .2
weight_user_median = .8
penalty = .05


#load CSV
train_data = {}
test_data = {}
ndata = 0
with open(train_file, 'r') as train_fh:
    ndata = sum(1 for row in train_fh) - 1
    
splitting = np.random.binomial(1, train_size, size=ndata)


In [169]:
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for i, row, in enumerate(train_csv):
        user   = row[0]
        artist = row[1]
        plays  = row[2]

        if splitting[i]: 
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = int(plays)
            
        else:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = int(plays)
              
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [170]:
user_mat = []
artist_mat = []

artist_plays = {}
for k in range(K):
    print "\r", k,
    user_mat.append({})
    artist_mat.append({})
    
    for user, user_data in train_data.iteritems():
        if not user in user_mat[k]:
            user_mat[k][user] = np.random.rand()
        for artist, plays in user_data.iteritems():
            if not artist in artist_plays:
                artist_plays[artist] = []
            artist_plays[artist].append(plays)
            if not artist in artist_mat[k]:
                artist_mat[k][artist] = np.random.rand()

print len(user_mat[1])


29 233286


In [171]:
artist_meds = {}
for artist in artist_plays.keys():
    artist_meds[artist] = np.median(np.array(artist_plays[artist]))

In [172]:
i = 10
test_artist = artist_plays.keys()[i]

print artist_meds[test_artist]


116.0


In [173]:
%%time
def predict(user, artist):
    return sum([user_mat[k][user] * artist_mat[k][artist] for k in range(K)])


def modify_prediction(user, artist, prediction):
    user_bias = user_medians[user] - global_median
    return w*prediction + (1-w)*(artist_meds[artist] + user_bias)

def modify_prediction2(user, artist, prediction):
    return (1 - weight_user_median)*prediction + weight_user_median*(user_medians[user])

def train(data):

    for i in range(iterations):
        for k in range(K):
            user_feature = user_mat[k]
            artist_feature = artist_mat[k]

            for j, (user, user_data) in enumerate(train_data.iteritems()):
                if j % 10000 == 0:
                    print "\r", i,k,j,

                for artist, plays in user_data.iteritems():
                    error = plays - predict(user, artist)
                    user_feature[user] += learn_rate * (artist_feature[artist] * error - penalty * user_feature[user])
                    artist_feature[artist] += learn_rate * (user_feature[user] * error - penalty * artist_feature[artist])
    print user_feature[user]

train(train_data)

9 29 230000 0.449389992043
Wall time: 4h 26min 24s
Parser   : 130 ms


In [175]:
def modify_prediction3(user, artist, prediction):
    user_bias = user_medians[user] - global_median
    return w1*user_medians[user] + w*(artist_meds[artist] + user_bias) + (1 - w - w1)*prediction

In [176]:
w = 0.3
w1 = 0.6
weight_user_median = 0.9

In [177]:
%%time
def calc_error(data):
    errors = []
    errors2 = []
    errors3 = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 10000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            pred = predict(user,artist)
                
            newpred = modify_prediction(user,artist,pred)
            error = abs(int(plays) - max(0, newpred))
            errors.append(error)

            newpred2 = modify_prediction2(user,artist,pred)
            error2 = abs(int(plays) - max(0, newpred2))
            errors2.append(error2)
            
            newpred3 = modify_prediction3(user,artist,pred)
            error3 = abs(int(plays) - max(0, newpred3))
            errors3.append(error3)


    return (sum(np.array(errors))/len(errors), sum(np.array(errors2))/len(errors2), sum(np.array(errors3))/len(errors3))
        
error = calc_error(test_data)

print "Error: ", error


190000 Error:  (146.09157283435212, 137.1111012963749, 137.07845895964059)
Wall time: 1min 17s


In [178]:
def calc_error2(data):
    errors2 = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 10000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            pred = predict(user,artist)
                
            newpred2 = modify_prediction2(user,artist,pred)
            error2 = abs(int(plays) - max(0, newpred2))
            errors2.append(error2)


    return sum(np.array(errors2))/len(errors2)

In [179]:
def calc_error13(data):
    errors = []
    errors3 = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 10000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            pred = predict(user,artist)
                
            newpred = modify_prediction(user, artist,pred)
            error = abs(int(plays) - max(0, newpred))
            errors.append(error)

            newpred3 = modify_prediction3(user,artist,pred)
            error3 = abs(int(plays) - max(0, newpred3))
            errors3.append(error3)


    return (sum(np.array(errors))/len(errors), sum(np.array(errors3))/len(errors3))

In [180]:
%%time
results2 = dict()
weight_user_median_params = np.hstack((np.arange(0, 1, 0.2), np.arange(0.9, 1.01, 0.01)))
for param in weight_user_median_params:
    print '\rtesting weight_user_median=%.2f' % param
    weight_user_median = param
    results2[param] = calc_error2(test_data)

testing weight_user_median=0.00
testing weight_user_median=0.20
testing weight_user_median=0.40
testing weight_user_median=0.60
testing weight_user_median=0.80
testing weight_user_median=0.90
testing weight_user_median=0.91
testing weight_user_median=0.92
testing weight_user_median=0.93
testing weight_user_median=0.94
testing weight_user_median=0.95
testing weight_user_median=0.96
testing weight_user_median=0.97
testing weight_user_median=0.98
testing weight_user_median=0.99
testing weight_user_median=1.00
190000Wall time: 2min 30s



In [104]:
import itertools

In [187]:
opt_param_2 = min(results2, key = lambda k : results2[k])
print weight_user_median_opt
print results2[weight_user_median_opt]

0.95
136.850259845


In [99]:
results2

{0.0: 241.14578422142853,
 0.10000000000000001: 224.94829780022167,
 0.20000000000000001: 208.9143517412499,
 0.30000000000000004: 193.24857193901636,
 0.40000000000000002: 178.36302798208956,
 0.5: 164.90095564633157,
 0.60000000000000009: 153.61040156472765,
 0.70000000000000007: 145.188481277839,
 0.80000000000000004: 139.86952387873768,
 0.90000000000000002: 137.38101931056315,
 0.91000000000000003: 137.26993288910745,
 0.92000000000000004: 137.18309392440844,
 0.93000000000000005: 137.1199917084607,
 0.94000000000000006: 137.08058595711569,
 0.95000000000000007: 137.0644552352037,
 0.96000000000000008: 137.0695556148213,
 0.97000000000000008: 137.09616449033186,
 0.98000000000000009: 137.14445965395086,
 0.9900000000000001: 137.21438562894423,
 1.0: 137.30438828414103}

In [120]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
plt.plot(results2.keys(), results2.values())

In [182]:
%%time
results13 = dict()
for weight_sum in [0.8, 0.85, 0.9, 0.95, 1]:
    for param in [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6]:
        w = param
        w1 = weight_sum - w
        print '\rtesting w=%.2f and w1=%.2f' % (w, w1)
        results13[(w, w1)] = calc_error13(test_data)

testing w=0.10 and w1=0.70
testing w=0.15 and w1=0.65
testing w=0.20 and w1=0.60
testing w=0.25 and w1=0.55
testing w=0.30 and w1=0.50
testing w=0.40 and w1=0.40
testing w=0.50 and w1=0.30
testing w=0.60 and w1=0.20
testing w=0.10 and w1=0.75
testing w=0.15 and w1=0.70
testing w=0.20 and w1=0.65
testing w=0.25 and w1=0.60
testing w=0.30 and w1=0.55
testing w=0.40 and w1=0.45
testing w=0.50 and w1=0.35
testing w=0.60 and w1=0.25
testing w=0.10 and w1=0.80
testing w=0.15 and w1=0.75
testing w=0.20 and w1=0.70
testing w=0.25 and w1=0.65
testing w=0.30 and w1=0.60
testing w=0.40 and w1=0.50
testing w=0.50 and w1=0.40
testing w=0.60 and w1=0.30
testing w=0.10 and w1=0.85
testing w=0.15 and w1=0.80
testing w=0.20 and w1=0.75
testing w=0.25 and w1=0.70
testing w=0.30 and w1=0.65
testing w=0.40 and w1=0.55
testing w=0.50 and w1=0.45
testing w=0.60 and w1=0.35
testing w=0.10 and w1=0.90
testing w=0.15 and w1=0.85
testing w=0.20 and w1=0.80
testing w=0.25 and w1=0.75
testing w=0.30 and w1=0.70
t

In [186]:
results1 = {k:v[0] for k,v in results13.items()}
results3 = {k:v[1] for k,v in results13.items()}

In [188]:
opt_param_1 = min(results1, key = lambda k : results1[k])
print opt_param_1
results1[opt_param_1]

(0.1, 0.7000000000000001)


140.0094723725112

In [191]:
print results3.keys()

[(0.6, 0.30000000000000004), (0.3, 0.6000000000000001), (0.15, 0.7), (0.5, 0.30000000000000004), (0.25, 0.55), (0.1, 0.7000000000000001), (0.5, 0.5), (0.15, 0.85), (0.4, 0.5), (0.2, 0.8), (0.2, 0.6499999999999999), (0.3, 0.5), (0.1, 0.8), (0.2, 0.75), (0.25, 0.7), (0.3, 0.55), (0.4, 0.5499999999999999), (0.25, 0.65), (0.1, 0.9), (0.25, 0.75), (0.4, 0.4), (0.5, 0.35), (0.6, 0.35), (0.5, 0.44999999999999996), (0.6, 0.4), (0.15, 0.65), (0.15, 0.7999999999999999), (0.6, 0.20000000000000007), (0.3, 0.6499999999999999), (0.1, 0.85), (0.25, 0.6), (0.4, 0.6), (0.4, 0.44999999999999996), (0.5, 0.4), (0.6, 0.25), (0.2, 0.7), (0.3, 0.7), (0.15, 0.75), (0.1, 0.75), (0.2, 0.6000000000000001)]


In [189]:
opt_param_3 = min(results3, key = lambda k : results3[k])
print opt_param_3
results3[opt_param_3]

(0.15, 0.7999999999999999)


136.62257934083038

In [190]:
print results1[opt_param_1]
print results2[opt_param_2]
print results3[opt_param_3]

140.009472373
136.850259845
136.622579341


In [202]:
params, results = zip(*results3.items())
w_vals, w1_vals = zip(*params)

In [207]:
np.append?

In [206]:
Axes3D.plot_trisurf(np.array(w_vals), np.array(w1_vals), np.array(results))

TypeError: unbound method plot_trisurf() must be called with Axes3D instance as first argument (got ndarray instance instead)

In [204]:
Axes3D.plot_trisurf?

In [121]:
Axes3D.plot_surface(w_vals, w1_vals, results)

TypeError: unbound method plot_surface() must be called with Axes3D instance as first argument (got tuple instance instead)

In [123]:
np.meshgrid([1,2,3], [1,2,3])

[array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]), array([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])]

In [73]:
import cPickle as pickle

In [195]:
with open('results2_new.p', 'w') as f:
    pickle.dump(results2, f)

In [196]:
with open('results13_new.p', 'w') as f:
    pickle.dump(results13, f)

In [174]:
%%time
with open('user_mat_NEW.p', 'w') as f:
    pickle.dump(user_mat, f)
with open('artist_mat_NEW.p', 'w') as f:
    pickle.dump(artist_mat, f)

Wall time: 5min 15s


In [ ]:
params.append([train_size, K, learn_rate, iterations, w, weight_user_median, penalty, error])

In [ ]:
print params

In [200]:
w, w1 = min(results3, key = lambda k : results3[k])
print w, w1

0.15 0.8


In [201]:
%%time
with open(test_file, 'r') as test_fh:
    with open('predictions_NEWEST.csv', 'w') as test_output:
        test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
        next(test_csv, None)
        writer = csv.writer(test_output)
        writer.writerow(('Id', 'plays'))

        for i, row in enumerate(test_csv):
            if i % 10000 == 0:
                print '\r', i,
            pred_id = row[0]
            user = row[1]
            artist = row[2]
            pred = predict(uaser,artist)
            newpred3 = modify_prediction3(user,artist,pred)
            writer.writerow((pred_id, max(0, newpred3)))

4150000Wall time: 3min 20s

